In [2]:
# Create RPE-PR combined model

from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [3]:
# adding back some reactions after CORDA

import pandas as pd

mod = models[k[0]].copy()

def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model

# add ATP demand reaction and lost exchange reactions
for k_i in k:
    models[k_i].add_reactions([mod.reactions.get_by_id('MAR03964').copy()])
    models[k_i] = add_all_EX_rxns(mod,models[k_i])
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (0,1000)     # close lower bound for all exchange reactions (metabolites can leave the system but not enter)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpoh4rtlzs.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


Ignoring reaction 'MAR03964' since it already exists.


In [4]:
# copy RPE, PR, and full Human1 models
mod = models['Human-GEM.xml'].copy()
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()
mod_Human1 = models['Human-GEM.xml'].copy()

# load Recon3D model
from cobra.io import load_matlab_model
import scipy
mod_Recon3D = load_matlab_model(Path().cwd() / 'cs_mods' / 'Recon3D_model2_corrected_subsystems.mat' )

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpujzmvo0h.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp3otzbcgt.lp
Reading time = 0.03 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp65ywabpm.lp
Reading time = 0.02 seconds
: 5013 rows, 12490 columns, 46478 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp0y7s65ix.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


No defined compartments in model model2. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


KeyboardInterrupt: 

In [5]:
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpo3y9ttcc.lp
Reading time = 0.03 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmppcawyw34.lp
Reading time = 0.02 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


In [6]:
mod_RPE

Name,RPE
Memory address,201a58a7e80
Number of metabolites,5517
Number of reactions,7097
Number of genes,2041
Number of groups,142
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Lysosome, Nucleus, Inner mitochondria"


In [7]:
mod_PR

Name,PR rod
Memory address,201a58a7a60
Number of metabolites,5013
Number of reactions,6245
Number of genes,2052
Number of groups,142
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Lysosome, Nucleus, Inner mitochondria"


In [4]:
# make RPE_PR model
def make_RPE_PR(mod_RPE,mod_PR):
    def remove_compartment(model, compartment):
    # remove compartment from model
        rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
        mets_n = [m for m in model.metabolites if compartment in m.compartment]
        model.remove_reactions(rxns_n)
        model.remove_metabolites(mets_n)
        return model

    # remove nucleus from PR model
    mod_PR = remove_compartment(mod_PR, 'n')

    def add_id_suffix(model, suffix):
        for r in model.reactions:
            r.id = r.id + suffix
        for m in model.metabolites:
            m.id = m.id + suffix
            m.compartment = m.compartment + suffix
        return model
    # add '_PR' to PR model IDs and '_RPE' to RPE model IDs
    mod_PR = add_id_suffix(mod_PR, '_PR')
    mod_RPE = add_id_suffix(mod_RPE, '_RPE')

    # add RPE_PR interface reactions to RPE model (rxn_RPE_PR = RPE <--> RPE_PR interface)
    for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
        rxn = rxns.copy()
        rxn.id = rxn.id.replace('_RPE','_RPE_PR')
        for p in rxn.products:
            if 'e_RPE' in p.id:
                p.id = p.id.replace('e_RPE','e_RPE_PR')
                p.compartment = 'e_RPE_PR'
        for r in rxn.reactants:
            if 'e_RPE' in r.id:
                r.id = r.id.replace('e_RPE','e_RPE_PR')
                r.compartment = 'e_RPE_PR'
        mod_RPE.add_reactions([rxn])

    # add RPE_PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE_PR interface)
    for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
        rxn.id = rxn.id.replace('_PR','_PR_RPE')
        for p in rxn.products:
            if 'e_PR' in p.id:
                p.id = p.id.replace('e_PR','e_RPE_PR')
                p.compartment = 'e_RPE_PR'
        for r in rxn.reactants:
            if 'e_PR' in r.id:
                r.id = r.id.replace('e_PR','e_RPE_PR')
                r.compartment = 'e_RPE_PR'

    # fuse RPE and PR model
    from cobra import Model
    # mod_RPE_PR = Model('RPE_PR model')
    mod_RPE_PR = mod_RPE.copy()
    mod_RPE_PR.id = 'RPE_PR'
    mod_RPE_PR.add_reactions(mod_PR.reactions)

    # list reaction IDs to be deleted (because their reactions are duplicated)
    import pandas as pd
    df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                       if r.compartments == {'e_RPE_PR'}], columns=['id','reaction'])
    l  = list(df['id'][df['reaction'].duplicated()])

    # delete duplicated reactions and change reaction IDs (suffix: _eRPE_PR) 
    # of the reactions that only involve the e_RPE_PR compartment (interface)
    for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE_PR'}]: 
        if r.id in l:
            mod_RPE_PR.remove_reactions(r.id)
        elif '_RPE_PR' in r.id:
            r.id = r.id.replace('_RPE_PR','_eRPE_PR')
        elif '_PR_RPE' in r.id:
            r.id = r.id.replace('_PR_RPE','_eRPE_PR')

    def fix_compartment_dict(model):
        # comment
        new_compartment_dict = model.compartments.copy()
        for k in list(new_compartment_dict.keys()):
            new_compartment_dict[k] = k 
        model.compartments = new_compartment_dict
        return model
    # add values to compartment dictionary
    fix_compartment_dict(mod_RPE_PR)

    def add_r2groups(model):
        # add all reactions to their subsystem group
        g_dict = {g.name:g.id for g in model.groups}
        for r in model.reactions:
            model.groups.get_by_any(g_dict[r.subsystem])[0].add_members([r])
    add_r2groups(mod_RPE_PR)
    
    mod_RPE_PR.id = mod_RPE.id + '_' mod_PR.id
    mod_RPE_PR.name = mod_RPE.id + '_' mod_PR.id    
    
    return mod_RPE_PR


mod_RPE_PR = make_RPE_PR(mod_RPE,mod_PR)
mod_Human1_Human1 = make_RPE_PR(mod_Human1.copy(),mod_Human1.copy())
mod_Recon3D_Recon3D = make_RPE_PR(mod_Recon3D.copy(),mod_Recon3D.copy())

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpic8z2lyb.lp
Reading time = 0.08 seconds
: 7182 rows, 19688 columns, 65842 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpq_pl9g07.lp
Reading time = 0.10 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpp43cwgxe.lp
Reading time = 0.11 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpnx8uxfkk.lp
Reading time = 0.17 seconds
: 10034 rows, 36120 columns, 143572 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpi9x3c3fd.lp
Reading time = 0.09 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpzcyg3s7g.lp
Reading time = 0.08 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmppeu26rx1.lp
Reading time = 0.10 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


In [10]:
from cobra.io import write_sbml_model, read_sbml_model
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")
mod_RPE_PR = read_sbml_model(Path().cwd() / 'mod_RPE_PR.xml')

Name,RPE_PR
Memory address,250d8dce350
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [11]:
from cobra.io import write_sbml_model, read_sbml_model
write_sbml_model(mod_Human1_Human1, "mod_Human1_Human1.xml")
mod_Human1_Human1 = read_sbml_model(Path().cwd() / 'mod_Human1_Human1.xml')

Name,RPE_PR
Memory address,250be4d0370
Number of metabolites,16554
Number of reactions,29057
Number of genes,3067
Number of groups,142
Objective expression,1.0*MAR13082_RPE - 1.0*MAR13082_RPE_reverse_4a4f4
Compartments,"c_RPE, e_RPE, l_RPE, r_RPE, m_RPE, x_RPE, g_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, m_PR, x_PR, r_PR, l_PR, g_PR, i_PR"


In [14]:
from cobra.io import write_sbml_model, read_sbml_model
write_sbml_model(mod_Recon3D_Recon3D, "mod_Recon3D_Recon3D.xml")
mod_Recon3D_Recon3D = read_sbml_model(Path().cwd() /'mod_Recon3D_Recon3D.xml')

Error encountered trying to <Setting notes on sbase: <Reaction R_ADCim_RPE "Acetoacetate Decarboxylation (Irreversible), Mitochondrial">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_CHOLtu_RPE "Choline Uniport">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_DEBRISOQUINEt_RPE "Debrisoquine Uniport">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a

Name,RPE_PR
Memory address,250d8dda2f0
Number of metabolites,11575
Number of reactions,21029
Number of genes,2248
Number of groups,105
Objective expression,1.0*biomass_maintenance_RPE - 1.0*biomass_maintenance_RPE_reverse_e118b
Compartments,"c_RPE, l_RPE, m_RPE, r_RPE, e_RPE, x_RPE, n_RPE, g_RPE, i_RPE, c_PR, l_PR, m_PR, r_PR, e_PR, x_PR, g_PR, i_PR"


In [25]:
mod_Recon3D_Recon3D = read_sbml_model(Path().cwd() / 'mod_Recon3D_Recon3D.xml')

In [26]:
mod_Recon3D_Recon3D

Name,RPE_PR
Memory address,251c8eacf40
Number of metabolites,11575
Number of reactions,21029
Number of genes,2248
Number of groups,105
Objective expression,1.0*biomass_maintenance_RPE - 1.0*biomass_maintenance_RPE_reverse_e118b
Compartments,"c_RPE, l_RPE, m_RPE, r_RPE, e_RPE, x_RPE, n_RPE, g_RPE, i_RPE, c_PR, l_PR, m_PR, r_PR, e_PR, x_PR, g_PR, i_PR"


In [7]:
# set objective function
mod_RPE_PR.objective = 'MAR03964_PR'
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Recon3D_Recon3D.objective = 'MAR03964_PR'

In [8]:
mod_RPE_PR

Name,RPE_PR
Memory address,250b5120640
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [9]:
mod_groups.reactions.get_by_id('MAR09364_eRPE_PR').subsystem

NameError: name 'mod_groups' is not defined

In [ ]:
mod_RPE_PR.reactions.get_by_id('MAR09364_eRPE_PR').subsystem

In [ ]:
mod_test = read_sbml_model('C:/Users/prins/GitHub/Human1_RPE-PR/mod_RPE_PR.xml')

In [ ]:
mod_test.reactions.get_by_id('MAR09364_eRPE_PR').subsystem

In [ ]:
# set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)  

# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

for i in range(len(bounds)): 
    models[k_i].reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)   

In [ ]:
# open RPE exchange reactions
for i in range(len(bounds)): 
    mod_RPE_PR.reactions.get_by_id(bounds['id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)   

In [ ]:
# ATP demand reactions RPE
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE')

In [ ]:
# ATP demand reactions PR
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

In [ ]:
# glucose exchange reaction RPE
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

In [ ]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('MAR03964_PR').flux_expression
solution = mod_RPE_PR.optimize()
print(solution)

In [ ]:
def get_vmh_id(reaction):
    vmh_id = ''
    if 'vmhreaction' in reaction.annotation.keys(): 
        vmh_id = reaction.annotation['vmhreaction']
    return  vmh_id
    
df= pd.DataFrame([[get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod_RPE_PR.reactions],\
                 index = [r.id  for r in mod_RPE_PR.reactions],
             columns=['vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
objectives = {mod_RPE_PR.reactions.get_by_id('MAR03964_PR'): 2, mod_RPE_PR.reactions.get_by_id('MAR03964_RPE'): 1}

from cobra.flux_analysis import flux_variability_analysis
mod_RPE_PR.objective = 'MAR03964_PR'
mod_RPE_PR.objective.expression
# fva = flux_variability_analysis(mod_RPE_PR, loopless=True)

In [ ]:
pd.merge(df, solution.to_frame(), left_index=True, right_index=True)

In [ ]:
solution.to_frame()

In [ ]:
#from cobra.flux_analysis.loopless import add_loopless, loopless_solution
#solution_loopless = loopless_solution(mod_RPE_PR)

#import pandas
#df = pandas.DataFrame(dict(loopless=solution_loopless.fluxes, nominal=solution.fluxes))

In [ ]:
df

In [ ]:
# 
#with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    solution_add_loopless = mod_RPE_PR.optimize()
#print("loopless solution: status = " + solution_add_loopless.status)
#print("loopless solution flux: v3 = %.1f" % solution_add_loopless.fluxes["v3"])
# solution_add_loopless.fluxes

In [ ]:
mod_RPE_PR.summary()

In [ ]:
solution_loopless

In [ ]:
from cobra.flux_analysis import pfba
solution_pfba = pfba(mod_RPE_PR)
print("parsimonious solution: status = " + solution_pfba.status)

In [ ]:
solution_pfba

In [ ]:
solution_pfba.fluxes

In [ ]:
solution_pfba.to_frame()

In [ ]:
s = mod_RPE_PR.summary(fva=0.95)

In [ ]:
solution.to_frame()

In [ ]:
s.to_frame()

In [ ]:
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

In [ ]:
# with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    try: solution = mod_RPE_PR.optimize()
#    except: print('model is infeasible')

In [ ]:
# FVA
#from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(mod_RPE_PR,loopless=True)

In [ ]:
# list all exchange and demand reactions matching Human1 and VMH IDs
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): vmh_id = reaction.annotation['vmhreaction']
    else: vmh_id = ''
    return vmh_id
df_EX_rxns = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products) == 0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns

In [ ]:
# ATP metabolites
ATP_mets = [[met.id,met.name] for met in mod_RPE_PR.metabolites if met.name=='ATP']
pd.DataFrame(ATP_mets,columns=['ID','name'])

# ATP reactions
ATP_rxns = [[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
            for r in mod_RPE_PR.reactions if 'MAM01371' in r.reaction]

pd.DataFrame(ATP_rxns,columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
# list all exchange and demand reactions
df_EX_rxns_human1 = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products)==0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns.to_clipboard(excel=True, sep=None)
df_EX_rxns

In [ ]:
# match VMH_IDs from bounds file to Human1 IDs to create 'R3D301_EX_rxns_RPE_PR.xlsx'

# import excel file with info on bounds
bounds = pd.read_excel(folder / 'rxn_bounds' / 'files PL' / 'R3D301_EX_rxns.xlsx')

# vmh_id lookup table + look up RPE rxn
df_vmh_id = pd.DataFrame([[get_vmh_id(r),r.id,r.id+'_RPE'] for r in mod.reactions if 'EX_' in get_vmh_id(r)],columns=['vmh_id','human1_id','id'])
merged_df = df_vmh_id.merge(bounds, how = 'right', on = ['vmh_id'])
merged_df.to_clipboard(excel=True, sep=None)
merged_df